# Topic Modeling
In this exercise, we will do topic modeling with gensim. Use the [topics and transformations tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html) as a reference.

In [12]:
import os
from collections import defaultdict

import gensim
import nltk

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

For tokenizing words and stopword removal, download the NLTK punkt tokenizer and stopwords list.

In [14]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

First, we load the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included with gensim that contains 300 news articles of the Australian Broadcasting Corporation.

In [15]:
from gensim.test.utils import datapath
train_file = datapath('lee_background.cor')
articles_orig = open(train_file).read().splitlines()

Preprocess the text by lowercasing, removing stopwords, stemming, and removing rare words.

In [16]:
# define stopword list
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords | {'\"', '\'', '\'\'', '`', '``', '\'s'}

# initialize stemmer
stemmer = nltk.stem.PorterStemmer()

def preprocess(article, stem=False):
    # tokenize
    article = nltk.word_tokenize(article)

    # lowercase all words
    article = [word.lower() for word in article]

    # remove stopwords
    article = [word for word in article if word not in stopwords]

    # optional: stem
    if stem:
        article = [stemmer.stem(word) for word in article]
    return article

articles = [preprocess(article) for article in articles_orig]

# create the dictionary and corpus objects that gensim uses for topic modeling
dictionary = gensim.corpora.Dictionary(articles)

# remove words that occur in less than 2 documents, or more than 50% of documents
dictionary.filter_extremes(no_below=2, no_above=0.5)
temp = dictionary[0]  # load the dictionary by calling it once
corpus_bow = [dictionary.doc2bow(article) for article in articles]

2025-02-28 09:12:38,210 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2025-02-28 09:12:38,233 : INFO : built Dictionary<7349 unique tokens: ["'ve", ',', '.', '100', '4:00pm']...> from 300 documents (total 40467 corpus positions)
2025-02-28 09:12:38,234 : INFO : Dictionary lifecycle event {'msg': 'built Dictionary<7349 unique tokens: ["\'ve", \',\', \'.\', \'100\', \'4:00pm\']...> from 300 documents (total 40467 corpus positions)', 'datetime': '2025-02-28T09:12:38.234297', 'gensim': '4.3.3', 'python': '3.9.21 (main, Dec 11 2024, 16:24:11) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'created'}
2025-02-28 09:12:38,238 : INFO : discarding 3829 tokens: [(',', 294), ('.', 300), ('associated', 1), ('burn', 1), ('claire', 1), ('cranebrook', 1), ('deterioration', 1), ('directions', 1), ('falls', 1), ('finger', 1)]...
2025-02-28 09:12:38,239 : INFO : keeping 3520 tokens which were in no less than 2 and no more than 150 


Now we create a TF-IDF model and transform the corpus into TF-IDF vectors.

In [17]:
from gensim import models


tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

print(corpus_bow[0])
print(corpus_tfidf[0])

2025-02-28 09:12:38,264 : INFO : collecting document frequencies
2025-02-28 09:12:38,265 : INFO : PROGRESS: processing document #0
2025-02-28 09:12:38,274 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 300 documents and 3520 features (22934 matrix non-zeros)', 'datetime': '2025-02-28T09:12:38.274473', 'gensim': '4.3.3', 'python': '3.9.21 (main, Dec 11 2024, 16:24:11) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'initialize'}


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 7), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 2), (49, 2), (50, 3), (51, 3), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 8), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 5), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 3), (99, 1), (100, 1), (101, 3), (102, 1), (103, 1), (104, 1), (105, 4), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)]

Now we train an [LDA model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html) with 10 topics on the TF-IDF corpus. Save it to a variable `model_lda`.

In [18]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50

# Make an index to word dictionary.
id2word = dictionary.id2token

model_lda = LdaModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    num_topics=num_topics,
)

2025-02-28 09:12:38,283 : INFO : using symmetric alpha at 0.02
2025-02-28 09:12:38,284 : INFO : using symmetric eta at 0.02
2025-02-28 09:12:38,285 : INFO : using serial LDA version on this node
2025-02-28 09:12:38,298 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 300 documents, updating model once every 300 documents, evaluating perplexity every 300 documents, iterating 50x with a convergence threshold of 0.001000
2025-02-28 09:12:38,299 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2025-02-28 09:12:38,516 : INFO : -203.990 per-word bound, 25528097335895521396288743951455578835329263826604725240856576.0 perplexity estimate based on a held-out corpus of 300 documents with 2210 words
2025-02-28 09:12:38,518 : INFO : PROGRESS: pass 0, at document #300/300
2025-02-28 09:12:38,776 : INFO : topic #7 (0.020): 0.005*"markets" + 0.004*"consumers" + 0.004*

Let's inspect the first 5 topics of our model.

In [19]:
model_lda.print_topics(5)

2025-02-28 09:12:38,800 : INFO : topic #45 (0.020): 0.005*"suharto" + 0.005*"ses" + 0.004*"wind" + 0.004*"doctors" + 0.003*"blue" + 0.003*"condition" + 0.003*"hour" + 0.003*"received" + 0.003*"warning" + 0.003*"mountains"
2025-02-28 09:12:38,803 : INFO : topic #31 (0.020): 0.003*"claim" + 0.003*"places" + 0.003*"palestinian" + 0.003*"hectares" + 0.003*"darwin" + 0.003*"native" + 0.002*"title" + 0.002*"weapons" + 0.002*"plants" + 0.002*"abc"
2025-02-28 09:12:38,811 : INFO : topic #49 (0.020): 0.005*"state" + 0.004*"south" + 0.003*"mounting" + 0.003*"india" + 0.003*"economy" + 0.003*"detainees" + 0.003*"test" + 0.003*"sydney" + 0.003*"industrial" + 0.003*"phillips"
2025-02-28 09:12:38,816 : INFO : topic #40 (0.020): 0.005*"space" + 0.004*"endeavour" + 0.004*"japanese" + 0.004*"report" + 0.003*"station" + 0.003*"boat" + 0.003*"crew" + 0.003*"shuttle" + 0.003*"company" + 0.003*"board"
2025-02-28 09:12:38,831 : INFO : topic #7 (0.020): 0.005*"markets" + 0.004*"consumers" + 0.004*"man" + 0.0

[(45,
  '0.005*"suharto" + 0.005*"ses" + 0.004*"wind" + 0.004*"doctors" + 0.003*"blue" + 0.003*"condition" + 0.003*"hour" + 0.003*"received" + 0.003*"warning" + 0.003*"mountains"'),
 (31,
  '0.003*"claim" + 0.003*"places" + 0.003*"palestinian" + 0.003*"hectares" + 0.003*"darwin" + 0.003*"native" + 0.002*"title" + 0.002*"weapons" + 0.002*"plants" + 0.002*"abc"'),
 (49,
  '0.005*"state" + 0.004*"south" + 0.003*"mounting" + 0.003*"india" + 0.003*"economy" + 0.003*"detainees" + 0.003*"test" + 0.003*"sydney" + 0.003*"industrial" + 0.003*"phillips"'),
 (40,
  '0.005*"space" + 0.004*"endeavour" + 0.004*"japanese" + 0.004*"report" + 0.003*"station" + 0.003*"boat" + 0.003*"crew" + 0.003*"shuttle" + 0.003*"company" + 0.003*"board"'),
 (7,
  '0.005*"markets" + 0.004*"consumers" + 0.004*"man" + 0.004*"powell" + 0.004*"companies" + 0.004*"food" + 0.004*"williams" + 0.004*"pilot" + 0.003*"1998" + 0.003*"genetically"')]

We see the 5 topics with the highest importance. For each topic, the 10 most important words are shown, together with their coefficient of "alignment" to the topic.

## Document Similarity
We now use our LDA model to compare the similarity of new documents (*queries*) to documents in our collection.

First, create an index of the news articles in our corpus. Use the `MatrixSimilarity` transformation as described in gensim's [similarity queries tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html).

In [20]:
from gensim import similarities
index = similarities.MatrixSimilarity(model_lda[corpus_tfidf])

2025-02-28 09:12:38,843 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2025-02-28 09:12:39,129 : INFO : creating matrix with 300 documents and 50 features


Now, write a function that takes a query string as input and returns the LDA representation for it. Make sure to apply the same preprocessing as we did to the documents.

In [21]:
def get_lda_representation(query_preprocessed):
    query_preprocessed = preprocess(query_preprocessed)
    query_bow = dictionary.doc2bow(query_preprocessed)
    query_tfidf = tfidf[query_bow]
    query_lda = model_lda[query_tfidf]
    return query_lda

query = "A new bill sparked massive protests in Israel, as it would massively limit the powers of the judiciary."
print(get_lda_representation(query))

[(21, 0.3230583), (35, 0.15599045), (44, 0.17489156)]


Print the top 5 most similar documents, together with their similarities, using your index created above.

In [11]:
query_lda = get_lda_representation(query)
sims = index[query_lda]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims[:5]:
    print(f"Similarity: {doc_score}, Article: {articles_orig[doc_position]}")

Similarity: 0.8096423745155334, Article: Russian authorities have sentenced Chechen warlord Salman Raduyev to life in prison for a 1996 hostage siege in which more than 200 people died. Salman Raduyev is probably the most important Chechen fighter Russian authorities have ever caught. A relative of the first Chechen president, he was at the forefront of the insurgency leading raids against federal troops. He was jealous of the achievements of his fellow commanders. He resolved to outperform his rival and in January 1996, masterminded a hostage taking in the neighbouring republic of Dagestan. Apparently, the aim was to destabilise Dagestan and spread the war to the rest of the Caucuses. He ran out of luck as Russian solders were not prepared to negotiate and cornered Raduyev on the Chechen border. 
Similarity: 0.8096423745155334, Article: Kashmiri militant groups denied involvement in Thursday's attack on the Indian Parliament, accusing Indian intelligence instead. "We want to make it c

Run your code again, now training an LDA model with 100 topics. Do you see a qualitative difference in the top-5 most similar documents?